In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)

In [5]:
tool.invoke({"query": "What happened at the last wimbledon"})

[{'url': 'https://www.theguardian.com/sport/live/2023/jul/16/wimbledon-mens-singles-final-2023-carlos-alcaraz-v-novak-djokovic-live?page=with:block-64b3ff568f08df28470056bf',
  'content': 'Carlos Alcaraz recovered from a set down to topple Djokovic 1-6, 7-6(6), 6-1, 3-6, 6-4 and win his first Wimbledon title in a battle for the ages'},
 {'url': 'https://www.bbc.com/sport/tennis/articles/c4ngz77j5y5o',
  'content': 'Making his return to Wimbledon after being injured himself last year, Draper celebrated clinching victory at 21:15 BST by whacking a ball into the jubilant crowd.'},
 {'url': 'https://sports.yahoo.com/live/wimbledon-mens-semifinals-carlos-alcaraz-novak-djokovic-set-to-face-off-in-wimbledon-final-rematch-121511865.html',
  'content': "Carlos Alcaraz defeated Daniil Medvedev in Friday's first men's semifinal at Wimbledon, 6-7, 6-3, 6-4, 6-4. It's the second straight year that Alcaraz has knocked Medvedev out in the semifinals at ..."},
 {'url': 'https://apnews.com/article/wimb

In [6]:
from langchain_core.tools import tool
@tool
def TavilySearchAPIWrapperTool(query:str): 
    """Tool to perform internet search  using Tavily search engine"""
    tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    # description="...",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)
    return tool.invoke({"query": query})


In [7]:
from langchain_openai import ChatOpenAI

tools = [TavilySearchAPIWrapperTool]

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [8]:
from langchain import hub
from langchain.agents import create_tool_calling_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

agent = create_tool_calling_agent(llm, tools, prompt)

c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [9]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
# query = "What is the current exchange rate from USD to JPY?"

# query = "get time series weekly data for Tesla?"

# query = "get daily  data for Tesla?"

# query = "get quote for Tesla?"

query = "Tell me latest hot news?"


# response = agent_executor.invoke(query)

agent_executor.invoke({"input": query})




> Entering new AgentExecutor chain...

Invoking: `TavilySearchAPIWrapperTool` with `{'query': 'latest news'}`


[{'url': 'https://www.cnn.com/', 'content': 'View the latest news and breaking news today for U.S., world, weather, entertainment, politics and health at CNN.com.'}, {'url': 'https://apnews.com/', 'content': 'In a political shift to the far right, anti-Islam populist Geert Wilders wins big in Dutch election\nEurope’s far-right populists buoyed by Wilders’ win in Netherlands, hoping the best is yet to come\nDaniel Noboa is sworn in as Ecuador’s president, inheriting the leadership of a country on edge\nOn the cusp of climate talks, UN chief Guterres visits crucial Antarctica\nBUSINESS\nOpenAI brings back Sam Altman as CEO just days after his firing unleashed chaos\nThis week’s turmoil with ChatGPT-maker OpenAI has heightened trust concerns in the AI world\nTo save the climate, the oil and gas sector must slash planet-warming operations, report says\nArgentina’s labor leaders

{'input': 'Tell me latest hot news?',
 'output': "Here are some of the latest hot news stories:\n\n1. **Geert Wilders Wins Big in Dutch Election**: In a significant political shift to the far right, anti-Islam populist Geert Wilders has achieved a notable victory in the Netherlands, with hopes among Europe’s far-right populists for further success. [Read more here](https://apnews.com/).\n\n2. **OpenAI's Turmoil**: Sam Altman has been reinstated as CEO of OpenAI just days after his controversial firing, a move that has heightened trust concerns in the AI community. [More details here](https://apnews.com/).\n\n3. **Ecuador's New Leadership**: Daniel Noboa has been sworn in as Ecuador’s president amidst a challenging political landscape. [Learn more about this story](https://apnews.com/).\n\n4. **Climate Crisis Developments**: A report indicates that the oil and gas sector must drastically reduce operations to combat climate change effectively. [Explore the findings here](https://apnews.c